In [7]:
# Classify movie reviews into positive or negative

In [6]:
import pandas as pd

df = pd.read_csv('../data/imdb-review-dataset/imdb_master.csv',
                 encoding='latin_1', index_col=0)

df1 = df.loc[df.label != 'unsup']

df1.label.value_counts() # count each label value

pos    25000
neg    25000
Name: label, dtype: int64

In [7]:
df_train = df1.loc[df1.type == 'train']
df_test = df1.loc[df1.type == 'test']

In [8]:
df_train = df_train.sample(n=2000, random_state=42)
df_test = df_test.sample(n=2000, random_state=42)

In [28]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\issohl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
from nltk import word_tokenize # tokenization
from nltk.stem import WordNetLemmatizer # lemmatizes
from sklearn.feature_extraction.text import TfidfVectorizer # computes tfidf

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()

    def __call__(self, doc):

        # tokenize text into tokens
        tokens = word_tokenize(doc)        

        # strip out punctuation
        words = [t for t in tokens if t.isalpha()]
        
        # lemmatize each token
        return [self.wnl.lemmatize(t) for t in words]


tfidf = TfidfVectorizer(stop_words='english', analyzer='word',
                        tokenizer=LemmaTokenizer())

V_train = tfidf.fit_transform(df_train.review)
V_train

<2000x21460 sparse matrix of type '<class 'numpy.float64'>'
	with 165017 stored elements in Compressed Sparse Row format>

In [10]:
y_train = df_train.label

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# visualize only the first n rows
n = 300

tsne = TSNE(n_components=2)
Z_train_2d = tsne.fit_transform(V_train[:n].todense())
y_train = df_train.label[:n]

fig, ax = plt.subplots()

labels = ['pos', 'neg']

for l in labels:
    ax.scatter(Z_train_2d[y_train == l, 0],
               Z_train_2d[y_train == l, 1],
               label=l, alpha=.2) # alpha = transparency

ax.legend()
plt.show()

```
Stage 1: Setup
scaler = StandardScaler()
scaler.fit(X_train) # setup scaler with X_train's mean, std

Stage 2: Apply
scaler.transform(X_train) # scaler is unchanged, just performs transformation
scaler.transform(X_test) # scaler is unchanged, just performs transformation
scaler.transform(new_data) # scaler is unchanged, just performs transformation
...

Stage 1: Setup
tfidf = TfidfVectorizer(..)
tfidf.fit(X_train) # setup tfidf with X_train's terms, documents

Stage 2: Apply
tfidf.transform(X_train) # tfidf is unchanged, just performs transformation
tfidf.transform(X_test) # tfidf is unchanged, just performs transformation
tfidf.transform(new_text) # tfidf is unchanged, just performs transformation
```


In [20]:
# transform V_test from text to vectors

V_test = tfidf.transform(df_test.review) # not fit_transform()
y_test = df_test.label
y_train = df_train.label

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

# scale
scaler = StandardScaler(with_mean=False)
V_train_sc = scaler.fit_transform(V_train) # fit(V_train), transform(V_train)
V_test_sc = scaler.transform(V_test)

# fit SGD
sgd = SGDClassifier(random_state=42)
sgd.fit(V_train_sc, y_train)
print(sgd.score(V_test_sc, y_test))

# fit SVC (GridSearch because this doesn't use gradient descent)
Cs = [.0001, .001, .01, .1, 1, 10]
gammas = [.0001, .001, .01, .1, 1, 10]
gs_svc = GridSearchCV(SVC(random_state=42), param_grid={'C':Cs, 'gamma':gammas})
gs_svc.fit(V_train_sc, y_train)
print(gs_svc.best_params_)
print(gs_svc.score(V_test_sc, y_test))

C:\Users\issohl\AppData\Local\Continuum\miniconda3\envs\mldds01\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.6495
{'C': 10, 'gamma': 0.0001}
0.6325


In [25]:
# metrics
from sklearn.metrics import classification_report, confusion_matrix

print('SGD')
pred_sgd = sgd.predict(V_test_sc)
print(classification_report(y_test, pred_sgd))
print(confusion_matrix(y_test, pred_sgd))

print('SVC')
pred_svc = gs_svc.predict(V_test_sc)
print(classification_report(y_test, pred_svc))
print(confusion_matrix(y_test, pred_svc))

SGD
             precision    recall  f1-score   support

        neg       0.67      0.65      0.66      1040
        pos       0.63      0.65      0.64       960

avg / total       0.65      0.65      0.65      2000

[[677 363]
 [338 622]]
SVC
             precision    recall  f1-score   support

        neg       0.59      0.97      0.73      1040
        pos       0.88      0.27      0.42       960

avg / total       0.73      0.63      0.58      2000

[[1004   36]
 [ 699  261]]


In [26]:
df_unsup = df.loc[df.label == 'unsup'] # unlabeled reviews

test = df_unsup.iloc[1:4].review # take 3 reviews
print(test)

# Tokenize .... Tfidf
V_test_doc = tfidf.transform(test)

# Scale
V_test_doc_sc = scaler.transform(V_test_doc)

# predict using SVC and SGD
print('SGD', sgd.predict(V_test_doc_sc))
print('SVC', gs_svc.predict(V_test_doc_sc))

50001    Take a low budget, inexperienced actors doubli...
50002    Everybody has seen 'Back To The Future,' right...
50003    Doris Day was an icon of beauty in singing and...
Name: review, dtype: object
SGD ['neg' 'neg' 'neg']
SVC ['neg' 'neg' 'pos']


In [27]:
test.values[2]

'Doris Day was an icon of beauty in singing and acting by her warm voice and genius acting in different movies obtained this film by her legend songs as (Iwill never stop loving you) with soft melody and warm lyrics by magic voice of Day.<br /><br />James Cagney was a villain of Hollywood by shark eyes and voice to send for audience the core of badness and evil characters as his profile in cinema.The producer choose previously Ava Gardener to be the hero of this film in-front Cagney but Cagney refused this choose because he said that Gardner not familiar with his role and he cheesed Day in it because of her fantastic abilities between singing and acting and she succeeded in it with Cagney.'

In [ ]:
# Example
# doc = df_train.iloc[0].review

# tokens = word_tokenize(doc) # needs nltk.download('punkt')
# print(tokens[:20])

# lem = WordNetLemmatizer() # needs nltk.download('wordnet')

# lemmatized = [lem.lemmatize(t) for t in tokens]
# print(lemmatized[:20])

In [18]:
# Keras embedding layer
# https://www.liip.ch/en/blog/sentiment-detection-with-keras-word-embeddings-and-lstm-deep-learning-networks

# Without GPU: conda install keras
# With GPU: conda install keras-gpu

In [33]:
V_train_sc.shape

(2000, 21460)

In [65]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense, Conv1D, MaxPool1D

n_vocab = 5000

tfidf = TfidfVectorizer(stop_words='english', analyzer='word',
                        tokenizer=LemmaTokenizer(),
                        max_features=n_vocab)

tfidf.fit(df_train.review)
V_train_keras = tfidf.transform(df_train.review)
V_test_keras = tfidf.transform(df_test.review)

scaler = StandardScaler(with_mean=False)
scaler.fit(V_train_keras)
V_train_keras_sc = scaler.transform(V_train_keras)
V_test_keras_sc = scaler.transform(V_test_keras)

y_train_keras = y_train.map({'pos': 1, 'neg': 0})
y_test_keras = y_test.map({'pos': 1, 'neg': 0})

In [68]:
embedding_vector_length = 50

model = Sequential()
model.add(Embedding(n_vocab, embedding_vector_length, input_length=n_vocab))
model.add(Conv1D(100, kernel_size=3, activation='relu', padding='valid'))
model.add(MaxPool1D())
model.add(GRU(35))
model.add(Dense(1, activation='sigmoid')) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 5000, 50)          250000    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 4998, 100)         15100     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 2499, 100)         0         
_________________________________________________________________
gru_3 (GRU)                  (None, 35)                14280     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 36        
Total params: 279,416
Trainable params: 279,416
Non-trainable params: 0
_________________________________________________________________


In [76]:
from keras.callbacks import EarlyStopping, TensorBoard
import time

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) 

earlystopping = EarlyStopping(patience=2)
tensorboard = TensorBoard(log_dir='./logs/text_gru/%d' % time.time(),
                          write_graph=False)

n = 1000
batch_size = 16
model.fit(V_train_keras_sc[:n], y_train_keras[:n],
          validation_data=(V_test_keras_sc[:n], y_test_keras[:n]),
          epochs=20, batch_size=batch_size,
          callbacks=[earlystopping, tensorboard])

Train on 1000 samples, validate on 1000 samples
Epoch 1/20
1000/1000 [==============================] - 119s 119ms/step - loss: 0.6928 - acc: 0.5110 - val_loss: 0.6930 - val_acc: 0.5120
Epoch 2/20
1000/1000 [==============================] - 116s 116ms/step - loss: 0.6927 - acc: 0.5280 - val_loss: 0.6958 - val_acc: 0.4930
Epoch 3/20
1000/1000 [==============================] - 117s 117ms/step - loss: 0.6871 - acc: 0.5270 - val_loss: 0.6928 - val_acc: 0.5200
Epoch 4/20
1000/1000 [==============================] - 116s 116ms/step - loss: 0.6804 - acc: 0.5240 - val_loss: 0.6986 - val_acc: 0.5260
Epoch 5/20
1000/1000 [==============================] - 116s 116ms/step - loss: 0.6737 - acc: 0.5520 - val_loss: 0.7054 - val_acc: 0.5240


In [49]:
model.evaluate(V_test_keras_sc, y_test_keras)

2000/2000 [==============================] - 83s 42ms/step


[0.69238569688797, 0.52]

In [50]:
test = df_unsup.iloc[1:10].review # take 3 reviews
print(test)

# Tokenize .... Tfidf
V_test_doc = tfidf.transform(test)

# Scale
V_test_doc_sc = scaler.transform(V_test_doc)

# predict
print('Keras Embedding layer', model.predict(V_test_doc_sc))

50001    Take a low budget, inexperienced actors doubli...
50002    Everybody has seen 'Back To The Future,' right...
50003    Doris Day was an icon of beauty in singing and...
50004    After a series of silly, fun-loving movies, 19...
50005    This isn't exactly a musical, but it almost se...
50006    After seven years and seventeen pictures at Wa...
50007    In the 1950's there were many film boigraphies...
50008    MY RATING- 7.3<br /><br />This one is a curiou...
50009    Doris Day and James Cagney are excellent in th...
Name: review, dtype: object
Keras Embedding layer [[0.48327655]
 [0.48329026]
 [0.48327655]
 [0.4832878 ]
 [0.4832897 ]
 [0.48354024]
 [0.48327655]
 [0.48327655]
 [0.48327655]]
